In [ ]:
import numpy as np
import re

from tensorflow.keras.layers import Dense, SimpleRNN, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer

text = """
• Ви - найкраще рішення для проблем, що виникли у понеділок.
• Думайте позитивно і вірте в свою здатність досягати відмінних результатів.
• Якщо ви змогли у понеділок піднятися з ліжка, це означає, що ви супергерой.
"""

# Обробка тексту
text = re.sub(r'[^\w\s]', '', text)  # видаляє всі символи крім букв та пробілів

# парсим текст, як послідовність символів
num_characters = 34  # 33 букви + пробіл
tokenizer = Tokenizer(num_words=num_characters, char_level=True)  # токенизуємо на рівні символів
tokenizer.fit_on_texts([text])  # формуємо токени на основі частотності в нашому тексті
print(tokenizer.word_index)

inp_chars = 3
data = tokenizer.texts_to_matrix(text)  # перетворюємо текст в масив
n = data.shape[0] - inp_chars  # так як ми передбачаємо по трьох символах - четвертий

X = np.array([data[i:i + inp_chars, :] for i in range(n)])
Y = data[inp_chars:]  # передбачення наступного символа

# архітектура моделі
model = Sequential()
model.add(Input((inp_chars, num_characters)))  # вказуємо два числа: довжину послідовності та розмір
model.add(SimpleRNN(128, activation='tanh'))  # рекурентний шар на 128 нейронів
model.add(Dense(num_characters, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Тренування моделі
history = model.fit(X, Y, batch_size=32, epochs=100)


def buildPhrase(inp_str, str_len=50):
    for i in range(str_len):
        x = []
        for j in range(i, i + inp_chars):
            x.append(tokenizer.texts_to_matrix(inp_str[j]))  # перетворюємо символи в One-Hot-encoding

        x = np.array(x)
        inp = x.reshape(1, inp_chars, num_characters)

        pred = model.predict(inp)  # передбачуємо  четвертого символа
        d = tokenizer.index_word[pred.argmax(axis=1)[0]]  # отримуємо відповідь в символьному вигляді

        inp_str += d  # дописуємо рядок

    return inp_str


res = buildPhrase("Якщ")
print(res)



{' ': 1, 'о': 2, 'и': 3, 'н': 4, 'е': 5, 'і': 6, 'в': 7, 'а': 8, 'т': 9, 'д': 10, 'л': 11, 'р': 12, 'к': 13, 'я': 14, 'п': 15, 'з': 16, 'у': 17, 'с': 18, '\n': 19, 'щ': 20, 'м': 21, 'й': 22, 'г': 23, 'ь': 24, 'ш': 25, 'б': 26, 'ю': 27, 'х': 28, 'ж': 29, 'ц': 30, 'ч': 31, 'є': 32}

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128)               20864     
                                                                 
 dense (Dense)               (None, 34)                4386      
                                                                 
Total params: 25250 (98.63 KB)
Trainable params: 25250 (98.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________

Epoch 1/100


7/7 [==============================] - 3s 9ms/step - loss: 3.5089 - accuracy: 0.0539 
Epoch 2/100
7/7 [===============